In [1]:
import numpy as np

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [4]:
from sklearn.model_selection import GridSearchCV

In [5]:
import pandas as pd

In [6]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [7]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [9]:
processed = pd.read_csv('features_01.csv')

In [10]:
processed = processed.set_index('person')
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel
person,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,14,6.0,Referral
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,0,16.0,Direct
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,66,10.0,Paid
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,5,20.6,Direct
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,2,17.0,Paid


In [11]:
predict_features = processed.loc[processed.index.isin(predict['person'])].drop(['primer_visita','ultima_visita'],axis=1)

In [12]:
training_features = processed.loc[processed.index.isin(a_predecir['person'])]
training_features['label'] = a_predecir.set_index('person')['label']


training_labels = training_features['label']
training_features = training_features.drop(['primer_visita','ultima_visita','label'],axis=1)

C:\Users\javif\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
training_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel
person,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,6,1,14,6.000000,Referral
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,17,1,2,17.000000,Paid
001802e4,ad campaign hit,iPhone 6S,Smartphone,0,1,4,19,1,0,19.000000,Paid
0019e639,viewed product,Motorola Moto G2 3G Dual,Computer,2,19,189,471,115,9,4.095652,Paid
001b0bf9,viewed product,Samsung Galaxy S8 Plus,Ninguno,1,0,2,7,1,9,7.000000,Ninguno


## Mean encoding

In [14]:
tg = category_encoders.TargetEncoder(smoothing=True)
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [15]:
encoded_training.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.002778,0.042654,0.068258,0,2,0,6,1,14,6.000000,0.066079
1,0.055313,0.050139,0.036301,0,1,3,17,1,2,17.000000,0.042803
2,0.031056,0.033590,0.036301,0,1,4,19,1,0,19.000000,0.042803
3,0.049121,0.030380,0.068258,2,19,189,471,115,9,4.095652,0.042803
4,0.049121,0.060150,0.017361,1,0,2,7,1,9,7.000000,0.017361


In [16]:
params={
    'n_estimators':[1,2,3,4,5,6,7,8,9,10]
}
grid = GridSearchCV(RandomForestClassifier(),params,scoring='roc_auc',cv=2,verbose=0)

In [17]:
xtr,xte,ytr,yte=train_test_split(encoded_training,training_labels)

In [18]:
grid.fit(xtr,ytr)

GridSearchCV(cv=2, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [19]:
grid.score(X=xte,y=yte)

0.6755033611272703

In [21]:
rfc = grid.best_estimator_
features = rfc.feature_importances_
pd.DataFrame([training_features.columns,features])

,0,1,2,3,4,5,6,7,8,9,10
0,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel
1,0.0328168,0.139821,0.0235538,0.031476,0.0763173,0.133801,0.138795,0.0905236,0.133156,0.154835,0.044905


## Binary Encoding (quitando producto mas visto porque tiene muchas categorias)

In [22]:
tg = category_encoders.BinaryEncoder()
encoded_training2 = tg.fit_transform(np.array(training_features.drop('prod_visto',axis=1)),np.array(training_labels))
encoded_predict2 = tg.transform(np.array(predict_features.drop('prod_visto',axis=1)))

In [23]:
encoded_training2.head()

,0_0,0_1,0_2,0_3,0_4,1_0,1_1,1_2,1_3,9_0,9_1,9_2,9_3,2,3,4,5,6,7,8
0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,2,0,6,1,14,6.000000
1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,3,17,1,2,17.000000
2,0,0,0,1,1,0,0,1,0,0,0,1,0,0,1,4,19,1,0,19.000000
3,0,0,1,0,0,0,0,0,1,0,0,1,0,2,19,189,471,115,9,4.095652
4,0,0,1,0,0,0,0,1,1,0,0,1,1,1,0,2,7,1,9,7.000000


In [24]:
params={
    'n_estimators':[1,2,3,4,5,6,7,8,9,10]
}
grid2 = GridSearchCV(RandomForestClassifier(),params,scoring='roc_auc',cv=2,verbose=0)

In [25]:
xtr,xte,ytr,yte=train_test_split(encoded_training2,training_labels)

In [26]:
grid2.fit(xtr,ytr)

GridSearchCV(cv=2, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [27]:
grid2.score(X=xte,y=yte)

0.6201186898067352

In [31]:
predictor = grid.best_estimator_

In [32]:
prediction_submit = predictor.predict_proba(encoded_predict)[:,1]

In [33]:
predict_features['label'] = prediction_submit

In [34]:
subm = predict.merge(predict_features['label'].reset_index(),how='left',on='person')

In [35]:
subm = subm.set_index('person')